In [1]:
import pandas as pd
from datetime import datetime


In [2]:
df = pd.read_csv('../data/current.csv')
df = df.drop(columns=['skaterFullName', 'teamAbbrevs', 'vs', 'shoots', 'row', 'date']) 
df = df.dropna()
df['avgPowerplayToi'] = df['avgPowerplayToi'].apply(lambda x: datetime.strptime(str(x), "%M:%S").second + datetime.strptime(str(x), "%M:%S").minute * 60)
df['position'].replace(['C', 'R', 'L', 'D'], [3, 2, 2, 1], inplace=True)
df.head()

,ppPoints,gamesPlayed,position,ppUnit,avgPowerplayToi,team_PEN/GP,team_PP%,team_PK%,team_G,team_GA,...,PPGF_enemy,PPG/PG_enemy,PPO/PG_enemy,PPTOI/PG_enemy,PEN_enemy,PENMin_enemy,PENSeconds/PG_enemy,PENDrawn/GP_enemy,PPGA_enemy,PKTIO/PG_enemy
2133,0,16,2,2.0,106,4.017515,0.15789,0.81176,74.0,90.0,...,18.0,0.75,3.29166,305.458333,110.0,258.0,645.0,4.029789,10.0,368.791666
2134,4,24,3,1.0,130,4.017515,0.15789,0.81176,74.0,90.0,...,18.0,0.75,3.29166,305.458333,110.0,258.0,645.0,4.029789,10.0,368.791666
2135,3,26,3,1.0,194,4.017515,0.15789,0.81176,74.0,90.0,...,18.0,0.75,3.29166,305.458333,110.0,258.0,645.0,4.029789,10.0,368.791666
2137,3,26,2,2.0,111,4.017515,0.15789,0.81176,74.0,90.0,...,18.0,0.75,3.29166,305.458333,110.0,258.0,645.0,4.029789,10.0,368.791666
2138,2,26,1,1.0,103,4.017515,0.15789,0.81176,74.0,90.0,...,18.0,0.75,3.29166,305.458333,110.0,258.0,645.0,4.029789,10.0,368.791666


In [3]:
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score

features = ['ppPoints', 'gamesPlayed', 'avgPowerplayToi', 'team_PEN/GP', 'team_PP%', 'team_PK%', 'team_GA', 'team_G', 'enemy_PEN/GP', 'enemy_PP%', 'enemy_PK%', 'enemy_GA', 'enemy_G']

features = ['ppPoints', 'gamesPlayed', 'position', 'ppUnit', 'avgPowerplayToi', 'team_G', 'team_GA', 'team_PP%', 'PPGF', 'PPG/PG', 'PPO/PG',
            'PPTOI/PG', 'PEN', 'PENMin', 'team_PEN/GP', 'PENSeconds/PG', 'PENDrawn/GP', 'PPGA', 'team_PK%', 'PKTIO/PG', 'enemy_G', 'enemy_GA', 'enemy_PP%',
            'PPGF_enemy', 'PPG/PG_enemy', 'PPO/PG_enemy', 'PPTOI/PG_enemy', 'PEN_enemy', 'PENMin_enemy', 'enemy_PEN/GP', 'PENSeconds/PG_enemy', 'PENDrawn/GP_enemy',
            'PPGA_enemy', 'enemy_PK%', 'PKTIO/PG_enemy']

label = ['scored']

X_train, X_test, y_train, y_test = train_test_split(df[features], df[label], test_size=0.2, random_state=42, shuffle=True)

from imblearn.over_sampling import RandomOverSampler
# undersampler = RandomUnderSampler(sampling_strategy='auto', random_state=42)
oversampler = RandomOverSampler(sampling_strategy='auto', random_state=42)

# Fit and transform the training data using the RandomUnderSampler
X_train, y_train = oversampler.fit_resample(X_train, y_train)


In [4]:
xgb_model = xgb.XGBClassifier(
    objective="binary:logistic", 
    eval_metric="logloss",
    n_estimators=1000,
    random_state=4
)

xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print(f'Accuracy: {acc}%')

Accuracy: 0.8238021638330757%


In [5]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.91      0.89       526
           1       0.53      0.45      0.49       121

    accuracy                           0.82       647
   macro avg       0.71      0.68      0.69       647
weighted avg       0.81      0.82      0.82       647

